In [2]:
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import time 
import pandas as pd
from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(options=chrome_options)
# webdriver.Chrome(options=chrome_options)

driver.maximize_window()

driver.get("https://www.google.com/maps/search/highland+coffee+qu%E1%BA%ADn+10/@10.7720455,106.6606587,15z/data=!3m1!4b1?entry=ttu")

number_star_list = []
name_list = []
review_time_list = []
review_cmt_list = []

In [3]:
start = 3
next_store = True

In [4]:
while next_store:
    try:
        store = driver.find_element(By.XPATH,f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[{start}]/div/a')
        store.click()
        time.sleep(3)
    except:
        next_store = False
        break

    review_section = driver.find_element(By.XPATH,'/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')
    review_section.click()
    time.sleep(3)
    scroll = True
    element = driver.find_element(By.XPATH,'/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')  # Replace 'element_id' with the actual ID of the element

    initial_height = driver.execute_script("return arguments[0].scrollHeight", element)

    while scroll:
        try:
            # Scroll the element down
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", element)
            
            # Wait for some time to let content load (adjust the time as needed)
            time.sleep(5)
            
            # Calculate the new height of the element
            new_height = driver.execute_script("return arguments[0].scrollHeight", element)
            
            # Break the loop if the element's height hasn't increased, indicating that we've reached the end
            if new_height == initial_height:
                scroll = False
            
            initial_height = new_height
        except:
            scroll = False

    current_html = driver.page_source
    soup = BeautifulSoup(current_html)
    soup.prettify()
    list_reviews = soup.find_all(class_ = ['jftiEf fontBodyMedium'])




    for review in list_reviews:
        number_star = review.find(class_ = 'kvMYJc')['aria-label']
        name_reviewer = review['aria-label']
        review_time = review.find('span',class_='rsqaWe').text

        review_cmt_element = review.find('span', class_='wiI7pd')
        if review_cmt_element:
            review_cmt = review_cmt_element.text
        else:
            review_cmt = "N/A"
        number_star_list.append(number_star)
        name_list.append(name_reviewer)
        review_time_list.append(review_time)
        review_cmt_list.append(review_cmt)
    start +=2

In [5]:
rev = { 'Reviewer Name': name_list,
        'Review Rate': number_star_list,
        'Review Time': review_time_list,
        'Review Comment':review_cmt_list
        }

df = pd.DataFrame(rev)

In [6]:
df

,Reviewer Name,Review Rate,Review Time,Review Comment
0,HuongCam Trinh,5 sao,một tháng trước,Quán này mình thấy ưng nhất trong chuỗi HL từ ...
1,Chin Chin,5 sao,4 tháng trước,Không gian thoáng mát nhiều chỗ ngồi. Mình ngồ...
2,Tchou Dung,5 sao,6 tháng trước,"Không gian rất thoáng nên không bị quá ồn ào, ..."
3,nguyen trinh,4 sao,2 tháng trước,"Quán gần nhà, không gian thoáng view nhìn ra đ..."
4,Anh Nguyễn Hoàng,5 sao,2 tháng trước,"Ở trong khu satra, có bãi xe, ké bóng cây lớn ..."
...,...,...,...,...
3733,Mai Linh Nguyễn,5 sao,2 tháng trước,N/A
3734,Hoàng Ân Dương,5 sao,3 tháng trước,N/A
3735,Justine BLOCH,5 sao,một tháng trước,N/A
3736,Светлана Статкевич,5 sao,một tháng trước,N/A


In [8]:
df.to_csv("data_reviews.csv")

In [7]:
time.sleep(5)

driver.close()